In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

from Vis import ClassificationVis, LabelVis
from Evals import *
from Data_Provider import *
from Util import ClassificationUtils as util
from Util import DataUtils as dutil


import numpy as np
from scipy import interp
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

/home/emil/miniconda3/envs/emocog/lib/python3.7/site-packages/hypertools/plot/__init__.py:10: UserWarning: Could not switch backend to TkAgg.  This may impact performance of the plotting functions.
  warnings.warn('Could not switch backend to TkAgg.  This may impact performance of the plotting functions.')
/home/emil/miniconda3/envs/emocog/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
# #important configs
# configs = dutil.load_configs()
# print(configs) #is this what we want?

In [4]:
# print(x.shape)
# print(y.shape)
# print(x_ev.shape)
# print(y_ev.shape)

# LabelVis.plot_happy_bars(y,y_ev)


In [5]:
# Run classifier with cross-validation
def calc_results_and_save(x,y,x_ev,y_ev, configs):
    cv = StratifiedKFold(n_splits=10,shuffle=True,random_state=1)
    results = pd.DataFrame(columns=('C','Gamma','Degree','AVG PR'))#,'AUC ROC'))
    degree=np.random.choice(np.arange(10)[1:],50)
    cs = np.random.uniform(0,4,50)
    gammas = 2**(np.random.choice(18,50)-15.)

    for idx,(deg,c,g) in enumerate(zip(degree,cs,gammas)):
        classifier = svm.SVC(cache_size=16000, C=c,kernel='poly', probability = True,gamma =g,degree=deg,random_state=5)
        auc_pr = util.get_auc_score(classifier, cv, x, y, go_after_pr=True)
        #auc_roc =util.get_auc_score(classifier, cv, x, y, go_after_pr=False)
        results.loc[idx] = [c,g,deg, auc_pr]
        print('Gamma = %.6f, Dim = %d, C= %.2f, Avg PR %.3f' %(g,deg,c,auc_pr))
    dutil.save_results(results,configs,'_poly') # save results for later use
    return results

def get_best_hyperparas(results):
    best_choice = util.get_best_hyperparas_results(results, col='Avg PR') #what were the best hyperparas?
    print(best_choice)
    best_c,best_g,best_d =best_choice[:3]
    return best_c,best_g,best_d

def vis_results(x,y, x_ev, y_ev, best_c, best_g, best_d):
    ###using hyperpara found, evaluate and get pretty plots
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

    classifier = svm.SVC(C=best_c,kernel='poly', probability = True,gamma =best_g,degree=best_d,random_state=5)
    best_thr = util.get_optimal_threshold(classifier, cv, x, y, go_after_pr=True) # get threshold using cv
    y_pred,y_pred_ev = util.fit_predict(classifier, best_thr, x, y, x_ev, y_ev) # using that threshold, get predictions and f1 score
    f1_tr=get_f1(y_pred,y) # calculate f1 scores for prediction on train set
    f1_ev=get_f1(y_pred_ev,y_ev)
    prec_tr,recall_tr = get_precision_recall(y_pred,y)
    prec_ev,recall_ev = get_precision_recall(y_pred_ev,y_ev)

    print(prec_tr, recall_tr,f1_tr)
    print(prec_ev, recall_ev,f1_ev)
    ClassificationVis.conf_mat(y_pred,y)
    ClassificationVis.conf_mat(y_pred_ev,y_ev)
    plt.show()

    ClassificationVis.plot_roc(x,y,classifier, 'SVC w/ Poly Kernel, degree= {:.2f}, C= {:.2f} Gamma = {:.2f}'.format(best_d,best_c, best_g))
    ClassificationVis.plot_roc(x_ev,y_ev,classifier, 'SVC w/ Poly Kernel, degree= {:.2f}, C= {:.2f} Gamma = {:.2f}'.format(best_d,best_c, best_g))
    ClassificationVis.plot_pr_curve(x,y,classifier, 'SVC w/ Poly Kernel, degree= {:.2f}, C= {:.2f} Gamma = {:.2f}'.format(best_d,best_c, best_g))
    ClassificationVis.plot_pr_curve(x_ev,y_ev,classifier,'SVC w/ Poly Kernel, degree= {:.2f}, C= {:.2f} Gamma = {:.2f}'.format(best_d,best_c, best_g))




In [6]:
%matplotlib inline

In [ ]:
files = [f for f in os.listdir('/home/emil/OpenMindv2/data') if f.endswith('hdf')]
provider = DataProvider()
for file in files:
    print('File',file)
    for cut in [.1,.2,.4]:
        print('Cut=',cut)
        configs = dutil.generate_configs_from_file(file, cut)
        x,y,x_ev,y_ev = provider.get_data(configs)
        print(x.shape)
        res = calc_results_and_save(x,y,x_ev,y_ev, configs)
#         a,b,c = get_best_hyperparas(res)
#         vis_results(x,y, x_ev, y_ev,a,b,c)

        

File ws_30_str_5_tr_s_0_e_30000_ev_s_30000_e_35000_expvar_90.hdf
Cut= 0.1
Loading Data from File..done
Doing cutoff
(5182, 99)
Gamma = 0.000244, Dim = 1, C= 2.46, Avg PR 0.550
Gamma = 0.007812, Dim = 7, C= 3.07, Avg PR 0.885
Gamma = 0.003906, Dim = 2, C= 2.89, Avg PR 0.937
Gamma = 0.003906, Dim = 2, C= 2.51, Avg PR 0.933
